# **Calculate Tensor Shape!**
Formula:
1. Convolution:
output dim = (input dim - filter size + 2 * padding)/stride + 1
<br>depth = # convolutions

In [1]:
import platform
platform.platform()

'macOS-10.16-x86_64-i386-64bit'

In [2]:
import torch
import torchvision.transforms as T
from torchvision.transforms import ToTensor
import numpy as np

In [3]:
H = 256
W = 232
C = 2
N = 1
image = np.zeros(shape=(H, W, C))
image_t = ToTensor()(image)
print(f"Shape of image_t, [C, H, W]: {image_t.shape}")
image_concat = torch.stack([image_t, image_t], dim=0)
print(f"Shape of image_concat, [N, C, H, W]: {image_concat.shape} where N = the first 2")
image_dummy = image_t[None,:] # adding dummy dimension!
print(f"Shape of image_concat, [N, C, H, W]: {image_dummy.shape} where N = the first 1")
image_0 = torch.zeros(C, H, W)
print(f"Shape of image_concat, [C, H, W]: {image_0.shape}")

Shape of image_t, [C, H, W]: torch.Size([2, 256, 232])
Shape of image_concat, [N, C, H, W]: torch.Size([2, 2, 256, 232]) where N = the first 2
Shape of image_concat, [N, C, H, W]: torch.Size([1, 2, 256, 232]) where N = the first 1
Shape of image_concat, [C, H, W]: torch.Size([2, 256, 232])


In [4]:
def convolution(tensor, count, filter, padding, stride):
    C = tensor.shape[0]
    H = tensor.shape[1]
    W = tensor.shape[2]
    new_C = int(count)
    new_H = int((H - filter + 2 * padding)/stride + 1)
    new_W = int((W - filter + 2 * padding)/stride + 1)
    new_image = torch.zeros((new_C, new_H, new_W))
    return new_image  
def transpose_convolution(tensor, count, filter, padding, stride):
    C = tensor.shape[0]
    H = tensor.shape[1]
    W = tensor.shape[2]
    new_C = int(count)
    new_H = int(stride * (H - 1) + filter - 2 * padding)
    new_W = int(stride * (W - 1) + filter - 2 * padding)
    # print(f"{new_W} = int({stride} * ({W} - 1) + {filter} - 2 * {padding})")
    new_image = torch.zeros((new_C, new_H, new_W))
    return new_image
def max_pooling(tensor, F, P, S):
    C = tensor.shape[0]
    return convolution(tensor=tensor, count=C, filter=F, padding=P, stride=S)

In [5]:
channels = 2
height = 256
width = 232
input = torch.zeros(channels, height, width)
input.shape

torch.Size([2, 256, 232])

In [195]:
A = torch.zeros(10, 1,2,3)
print(f"shape of A: {A.shape}")
B = torch.zeros(10, 1,4,5)
print(f"shape of B: {B.shape}")
C = T.CenterCrop(size=(A.shape[2], A.shape[3]))(B)
print(f"shape of C: {C.shape}")
D = torch.cat([A, C],dim=1)
print(f"shape of D: {D.shape}")

shape of A: torch.Size([10, 1, 2, 3])
shape of B: torch.Size([10, 1, 4, 5])
shape of C: torch.Size([10, 1, 2, 3])
shape of D: torch.Size([10, 2, 2, 3])


In [110]:
d1_1 = convolution(input, count=64, filter=3, padding=0, stride=1)
d1_2 = convolution(d1_1, count=64, filter=3, padding=0, stride=1)
print(f"Shape: [C, H, W]: {d1_2.shape}")
dt_1 = max_pooling(d1_2, 2, 0, 2)
print(f"Shape: [C, H, W]: {dt_1.shape}")

Shape: [C, H, W]: torch.Size([64, 568, 568])
Shape: [C, H, W]: torch.Size([64, 284, 284])


In [9]:
# neural network
print("going down")
d1_1 = convolution(input, count=64, filter=3, padding=0, stride=1) # <- contract 1
d1_2 = convolution(d1_1, count=64, filter=3, padding=0, stride=1)
print(f"Shape of contract 1: {d1_2.shape}, will concat")
dp1 = max_pooling(d1_2, 2, 0, 2)
d2_1 = convolution(dp1, count=128, filter=3, padding=0, stride=1) # <- contract 2
d2_2 = convolution(d2_1, count=128, filter=3, padding=0, stride=1)
print(f"Shape of contract 2: {d2_2.shape}, will concat")
dp2 = max_pooling(d2_2, 3, 0, 1)
db_1 = convolution(dp2, count=256, filter=3, padding=0, stride=1) # <- bottom
db_2 = convolution(db_1, count=256, filter=3, padding=0, stride=1)
print(f"Shape of BOTM: {db_2.shape}")
up1 = transpose_convolution(db_2, count=128, filter=10, padding=0, stride=2)
print(f"Shape of expand 1: {up1.shape}, for concat")
u1_1 = convolution(up1, count=128, filter=3, padding=0, stride=1) # <- expand 1
u1_2 = convolution(u1_1, count=128, filter=3, padding=0, stride=1)
up2 = transpose_convolution(u1_2, count=64, filter=18, padding=0, stride=2)
print(f"Shape of expand 2: {up1.shape}, for concat")
u2_1 = convolution(up2, count=64, filter=3, padding=0, stride=1) # <- expand 2
u2_2 = convolution(u2_1, count=64, filter=3, padding=0, stride=1)
out = convolution(u2_2, count=2, filter=1, padding=0, stride=1)
print(f"Shape of out: {out.shape}")

going down
Shape of contract 1: torch.Size([64, 252, 228]), will concat
Shape of contract 2: torch.Size([128, 122, 110]), will concat
Shape of BOTM: torch.Size([256, 116, 104])
Shape of expand 1: torch.Size([128, 240, 216]), for concat
Shape of expand 2: torch.Size([128, 240, 216]), for concat
Shape of out: torch.Size([2, 484, 436])


In [75]:
q = int((568 - 2 + 2 * 0)/2 + 1)
q


284